# 대리점 데이터로 데이터를 가공

In [1]:
import warnings
warnings.filterwarnings('ignore')

### 1 : 데이터 읽기

In [ ]:
# uriage.csv : 매출이력(기간 : 2019/1월~2019/7월)
# kokyaku_daicho.xlsx : 대리점에서 관리하는 고객 정보

In [4]:
# 입력 오류나, 표기 방법의 차이가 부정합을 일으킴 : 
    # 데이터의 정합성에 문제가 있음
import pandas as pd

uriage_data = pd.read_csv("./Data-in/uriage.csv")
uriage_data.head()

,purchase_date,item_name,item_price,customer_name
0,2019-06-13 18:02,상품A,100.0,김가온
1,2019-07-13 13:05,상 품 S,NaN,김우찬
2,2019-05-11 19:42,상 품 a,NaN,김유찬
3,2019-02-12 23:40,상품Z,2600.0,김재현
4,2019-04-22 3:09,상품a,NaN,김강현


In [3]:
kokyaku_data = pd.read_excel("./Data-in/kokyaku_daicho.xlsx")
kokyaku_data.head()

,고객이름,지역,등록일
0,김 현성,H시,2018-01-04 00:00:00
1,김 도윤,E시,42782
2,김 지한,A시,2018-01-07 00:00:00
3,김 하윤,F시,42872
4,김 시온,E시,43127


### 2 : 데이터 오류 확인

In [5]:
uriage_data["item_name"].head()

0      상품A
1    상 품 S
2    상 품 a
3      상품Z
4      상품a
Name: item_name, dtype: object

In [6]:
uriage_data["item_price"].head()

0     100.0
1       NaN
2       NaN
3    2600.0
4       NaN
Name: item_price, dtype: float64

In [7]:
kokyaku_data["등록일"].head()

0    2018-01-04 00:00:00
1                  42782
2    2018-01-07 00:00:00
3                  42872
4                  43127
Name: 등록일, dtype: object

### 3 : 데이터에 오류가 있는 상태로 집계

In [8]:
# '매출 이력'에서 상품별로 월 매출 합계를 집계
# index : purchase_month(년월만추출), columns : item_name, size

# 결과 : 원래 상품은 26개, 99개 출력으로 나옴

# 1. uriage_data["purchase_date"] -> date으로 변환
uriage_data["purchase_date"] = pd.to_datetime(uriage_data["purchase_date"])

# 2. starftime의 %Y%m 형식으로 변환
uriage_data["purchase_month"] = uriage_data["purchase_date"].dt.strftime("%Y%m")

# 3. 집계
res = uriage_data.pivot_table(index="purchase_month", columns="item_name", \
    aggfunc="size", fill_value=0)
res

item_name,상 품 n,상품 E,상품 M,상품 P,상품 S,상품 W,상품 X,상품W,상 품O,상 품Q,...,상품k,상품l,상품o,상품p,상품r,상품s,상품t,상품v,상품x,상품y
purchase_month,,,,,,,,,,,,,,,,,,,,,
201901,1,0,0,0,0,0,0,0,0,0,...,1,1,1,0,0,0,0,0,0,0
201902,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,1,1,1,0,0
201903,0,1,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
201904,0,0,0,0,0,0,0,1,0,1,...,0,0,0,0,0,1,0,0,0,0
201905,0,0,0,0,1,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,1
201906,0,0,0,0,0,1,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0
201907,0,0,0,0,0,0,0,0,1,0,...,0,0,1,0,2,0,0,0,0,0


In [9]:
# 매출 집계
res = uriage_data.pivot_table(index="purchase_month", columns="item_name", \
                              values="item_price", aggfunc="sum", fill_value=0)
res

item_name,상 품 n,상품 E,상품 M,상품 P,상품 S,상품 W,상품 X,상품W,상 품O,상 품Q,...,상품k,상품l,상품o,상품p,상품r,상품s,상품t,상품v,상품x,상품y
purchase_month,,,,,,,,,,,,,,,,,,,,,
201901,1400,0,0,0,0,0,0,0,0,0,...,1100,1200,1500,0,0,0,0,0,0,0
201902,0,0,0,0,0,0,2400,0,0,0,...,0,0,0,0,0,1900,2000,2200,0,0
201903,0,500,1300,1600,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
201904,0,0,0,0,0,0,0,2300,0,1700,...,0,0,0,0,0,1900,0,0,0,0
201905,0,0,0,0,1900,0,0,0,0,0,...,0,1200,0,0,0,0,0,0,0,2500
201906,0,0,0,0,0,2300,0,0,0,0,...,0,0,0,1600,0,0,0,0,2400,0
201907,0,0,0,0,0,0,0,0,0,0,...,0,0,1500,0,1800,0,0,0,0,0


### 4 : 상품명 오류 수정

In [10]:
# 상품 : A~Z까지 26개임
print(len(pd.unique(uriage_data["item_name"])))

99


In [11]:
# 상품명 오류 : 공백처리, 대소문자
uriage_data["item_name"] = uriage_data["item_name"].str.upper()
uriage_data["item_name"] = uriage_data["item_name"].str.replace(" ", "")
uriage_data.sort_values(by=["item_name"], ascending=True)

,purchase_date,item_name,item_price,customer_name,purchase_month
0,2019-06-13 18:02:00,상품A,100.0,김가온,201906
1748,2019-05-19 20:22:00,상품A,100.0,김시훈,201905
223,2019-06-25 08:13:00,상품A,100.0,김유진,201906
1742,2019-06-13 16:03:00,상품A,100.0,김건희,201906
1738,2019-02-10 00:28:00,상품A,100.0,김하랑,201902
...,...,...,...,...,...
2880,2019-04-22 00:36:00,상품Y,NaN,김동욱,201904
2881,2019-04-30 14:21:00,상품Y,NaN,김하준,201904
1525,2019-01-24 10:27:00,상품Y,2500.0,김범준,201901
1361,2019-05-28 13:45:00,상품Y,2500.0,김수현,201905


In [12]:
print(pd.unique(uriage_data["item_name"]))
print(len(pd.unique(uriage_data["item_name"])))

['상품A' '상품S' '상품Z' '상품V' '상품O' '상품U' '상품L' '상품C' '상품I' '상품R' '상품X' '상품G'
 '상품P' '상품Q' '상품Y' '상품N' '상품W' '상품E' '상품K' '상품B' '상품F' '상품D' '상품M' '상품H'
 '상품T' '상품J']
26


### 5 : 금액 결측치 수정

In [13]:
uriage_data.isnull().any(axis=0)

purchase_date     False
item_name         False
item_price         True
customer_name     False
purchase_month    False
dtype: bool

In [14]:
uriage_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2999 entries, 0 to 2998
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   purchase_date   2999 non-null   datetime64[ns]
 1   item_name       2999 non-null   object        
 2   item_price      2612 non-null   float64       
 3   customer_name   2999 non-null   object        
 4   purchase_month  2999 non-null   object        
dtypes: datetime64[ns](1), float64(1), object(3)
memory usage: 117.3+ KB


In [16]:
# test
for i, val in enumerate(['a', 'b', 'c']):
    print(i, val)

0 a
1 b
2 c


In [15]:
# test
# 특이사항 : 집계 기간에 상품 단가의 변동이 없다는 전제조건이 있음
flg_is_null = uriage_data["item_price"].isnull()
for idx, trg in enumerate(list(uriage_data.loc[flg_is_null, "item_name"].unique())):
    print(idx, trg)

0 상품S
1 상품A
2 상품P
3 상품N
4 상품W
5 상품R
6 상품I
7 상품L
8 상품F
9 상품O
10 상품B
11 상품C
12 상품V
13 상품Q
14 상품U
15 상품K
16 상품T
17 상품X
18 상품E
19 상품M
20 상품G
21 상품J
22 상품D
23 상품H
24 상품Y


In [17]:
# test
import pandas as pd

df = pd.DataFrame([{"country":"한국","population":500},{"country":"미국","population":450},{"country":"싱가폴","population":705},
                   {"country":"호주","population":878},{"country":"베트남","population":660},{"country":"대만","population":808}])
df

,country,population
0,한국,500
1,미국,450
2,싱가폴,705
3,호주,878
4,베트남,660
5,대만,808


In [18]:
# test
flg = df['country'] == '한국'
flg

0     True
1    False
2    False
3    False
4    False
5    False
Name: country, dtype: bool

In [19]:
df[flg] # test

,country,population
0,한국,500


In [20]:
df[~flg] # test

,country,population
1,미국,450
2,싱가폴,705
3,호주,878
4,베트남,660
5,대만,808


In [22]:
flg_is_null = uriage_data["item_price"].isnull()
# 결측치가 있는 상품명 리스트 작성
for trg in list(uriage_data.loc[flg_is_null, "item_name"].unique()):
    # print(trg)
    # item_name가 결측치가 없는 상품의 행이면서, 해당 상품명의 가격의 최대값
    # ~flg_is_null == (flg_is_null == False)
    price = uriage_data.loc[(~flg_is_null) & (uriage_data["item_name"] == trg),\
        "item_price"].max()
    uriage_data["item_price"].loc[(flg_is_null) & (uriage_data["item_name"]==\
        trg)] = price
uriage_data

,purchase_date,item_name,item_price,customer_name,purchase_month
0,2019-06-13 18:02:00,상품A,100.0,김가온,201906
1,2019-07-13 13:05:00,상품S,1900.0,김우찬,201907
2,2019-05-11 19:42:00,상품A,100.0,김유찬,201905
3,2019-02-12 23:40:00,상품Z,2600.0,김재현,201902
4,2019-04-22 03:09:00,상품A,100.0,김강현,201904
...,...,...,...,...,...
2994,2019-02-15 02:56:00,상품Y,2500.0,김정민,201902
2995,2019-06-22 04:03:00,상품M,1300.0,김재원,201906
2996,2019-03-29 11:14:00,상품Q,1700.0,김지율,201903
2997,2019-07-14 12:56:00,상품H,800.0,김승주,201907


In [23]:
uriage_data.isnull().any(axis=0)

purchase_date     False
item_name         False
item_price        False
customer_name     False
purchase_month    False
dtype: bool

In [24]:
for trg in list(uriage_data["item_name"].sort_values().unique()):
    print(trg + "의최고가：" + str(uriage_data.loc[uriage_data["item_name"]==\
        trg]["item_price"].max()) 
        + "의최저가：" + str(uriage_data.loc[uriage_data["item_name"]==trg]\
            ["item_price"].min(skipna=False)))

상품A의최고가：100.0의최저가：100.0
상품B의최고가：200.0의최저가：200.0
상품C의최고가：300.0의최저가：300.0
상품D의최고가：400.0의최저가：400.0
상품E의최고가：500.0의최저가：500.0
상품F의최고가：600.0의최저가：600.0
상품G의최고가：700.0의최저가：700.0
상품H의최고가：800.0의최저가：800.0
상품I의최고가：900.0의최저가：900.0
상품J의최고가：1000.0의최저가：1000.0
상품K의최고가：1100.0의최저가：1100.0
상품L의최고가：1200.0의최저가：1200.0
상품M의최고가：1300.0의최저가：1300.0
상품N의최고가：1400.0의최저가：1400.0
상품O의최고가：1500.0의최저가：1500.0
상품P의최고가：1600.0의최저가：1600.0
상품Q의최고가：1700.0의최저가：1700.0
상품R의최고가：1800.0의최저가：1800.0
상품S의최고가：1900.0의최저가：1900.0
상품T의최고가：2000.0의최저가：2000.0
상품U의최고가：2100.0의최저가：2100.0
상품V의최고가：2200.0의최저가：2200.0
상품W의최고가：2300.0의최저가：2300.0
상품X의최고가：2400.0의최저가：2400.0
상품Y의최고가：2500.0의최저가：2500.0
상품Z의최고가：2600.0의최저가：2600.0


### 6 : 고객이름 오류 수정

In [25]:
kokyaku_data["고객이름"].head()

0    김 현성
1    김 도윤
2    김 지한
3    김 하윤
4    김 시온
Name: 고객이름, dtype: object

In [26]:
uriage_data["customer_name"].head()

0    김가온
1    김우찬
2    김유찬
3    김재현
4    김강현
Name: customer_name, dtype: object

In [27]:
kokyaku_data["고객이름"] = kokyaku_data["고객이름"].str.replace(" ", "")
kokyaku_data["고객이름"].head()

0    김현성
1    김도윤
2    김지한
3    김하윤
4    김시온
Name: 고객이름, dtype: object

### 7 : 날짜오류를 수정하자

In [28]:
kokyaku_data["등록일"]

0      2018-01-04 00:00:00
1                    42782
2      2018-01-07 00:00:00
3                    42872
4                    43127
              ...         
195    2017-06-20 00:00:00
196    2018-06-20 00:00:00
197    2017-04-29 00:00:00
198    2019-04-19 00:00:00
199    2019-04-23 00:00:00
Name: 등록일, Length: 200, dtype: object

In [29]:
flg_is_serial = kokyaku_data["등록일"].astype("str").str.isdigit()
flg_is_serial.sum()

22

In [30]:
flg_is_serial

0      False
1       True
2      False
3       True
4       True
       ...  
195    False
196    False
197    False
198    False
199    False
Name: 등록일, Length: 200, dtype: bool

In [31]:
fromSerial = pd.to_timedelta(kokyaku_data.loc[flg_is_serial, "등록일"].\
    astype("float"), unit="D") + pd.to_datetime("1900/01/01")
fromSerial

1     2017-02-18
3     2017-05-19
4     2018-01-29
21    2017-07-06
27    2017-06-17
47    2017-01-08
49    2017-07-15
53    2017-04-10
76    2018-03-31
80    2018-01-12
99    2017-06-01
114   2018-06-05
118   2018-01-31
122   2018-04-18
139   2017-05-27
143   2017-03-26
155   2017-01-21
172   2018-03-24
179   2017-01-10
183   2017-07-26
186   2018-07-15
192   2018-06-10
Name: 등록일, dtype: datetime64[ns]

In [32]:
fromString = pd.to_datetime(kokyaku_data.loc[~flg_is_serial, "등록일"])
fromString

0     2018-01-04
2     2018-01-07
5     2017-06-20
6     2018-06-11
7     2017-05-19
         ...    
195   2017-06-20
196   2018-06-20
197   2017-04-29
198   2019-04-19
199   2019-04-23
Name: 등록일, Length: 178, dtype: datetime64[ns]

In [33]:
kokyaku_data["등록일"] = pd.concat([fromSerial, fromString])
kokyaku_data

,고객이름,지역,등록일
0,김현성,H시,2018-01-04
1,김도윤,E시,2017-02-18
2,김지한,A시,2018-01-07
3,김하윤,F시,2017-05-19
4,김시온,E시,2018-01-29
...,...,...,...
195,김재희,G시,2017-06-20
196,김도영,E시,2018-06-20
197,김이안,F시,2017-04-29
198,김시현,H시,2019-04-19


In [34]:
kokyaku_data["등록연월"] = kokyaku_data["등록일"].dt.strftime("%Y%m")
rslt = kokyaku_data.groupby("등록연월")["고객이름"].count()
print(rslt)
print(len(kokyaku_data))

등록연월
201701    15
201702    11
201703    14
201704    15
201705    13
201706    14
201707    17
201801    13
201802    15
201803    17
201804     5
201805    19
201806    13
201807    17
201904     2
Name: 고객이름, dtype: int64
200


In [35]:
# 등록일 컬럼에 숫자 데이터가 남아 있는지 확인
flg_is_serial = kokyaku_data["등록일"].astype("str").str.isdigit()
flg_is_serial.sum()

0

### 8 : 고객이름을 키로 두개의 데이터를 결합(조인)

In [38]:
# uriage_data, kokyaku_data

In [39]:
uriage_data.head()


,purchase_date,item_name,item_price,customer_name,purchase_month
0,2019-06-13 18:02:00,상품A,100.0,김가온,201906
1,2019-07-13 13:05:00,상품S,1900.0,김우찬,201907
2,2019-05-11 19:42:00,상품A,100.0,김유찬,201905
3,2019-02-12 23:40:00,상품Z,2600.0,김재현,201902
4,2019-04-22 03:09:00,상품A,100.0,김강현,201904


In [40]:
kokyaku_data.head()

,고객이름,지역,등록일,등록연월
0,김현성,H시,2018-01-04,201801
1,김도윤,E시,2017-02-18,201702
2,김지한,A시,2018-01-07,201801
3,김하윤,F시,2017-05-19,201705
4,김시온,E시,2018-01-29,201801


In [41]:
join_data = pd.merge(uriage_data, kokyaku_data, left_on="customer_name",\
    right_on="고객이름", how="left")
join_data = join_data.drop("customer_name", axis=1)
join_data

,purchase_date,item_name,item_price,purchase_month,고객이름,지역,등록일,등록연월
0,2019-06-13 18:02:00,상품A,100.0,201906,김가온,C시,2017-01-26,201701
1,2019-07-13 13:05:00,상품S,1900.0,201907,김우찬,C시,2018-04-07,201804
2,2019-05-11 19:42:00,상품A,100.0,201905,김유찬,A시,2018-06-19,201806
3,2019-02-12 23:40:00,상품Z,2600.0,201902,김재현,D시,2018-07-22,201807
4,2019-04-22 03:09:00,상품A,100.0,201904,김강현,D시,2017-06-07,201706
...,...,...,...,...,...,...,...,...
2994,2019-02-15 02:56:00,상품Y,2500.0,201902,김정민,B시,2017-07-01,201707
2995,2019-06-22 04:03:00,상품M,1300.0,201906,김재원,E시,2018-03-31,201803
2996,2019-03-29 11:14:00,상품Q,1700.0,201903,김지율,B시,2017-03-15,201703
2997,2019-07-14 12:56:00,상품H,800.0,201907,김승주,E시,2018-07-15,201807


### 9 : 정제 데이터 덤프

### 10 : 데이터를 집계